In [2]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.memory import ConversationSummaryBufferMemory
from langchain.schema.runnable import RunnablePassthrough

llm = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)

examples = [
    {
        "movie": "Top Gun",
        "answer": """
        🛩️👨‍✈️🔥
        """,
    },
    {
        "movie": "The Godfather",
        "answer": """
        👨‍👨‍👦🔫🍝
        """,
    },
    {
        "movie": "The Martian",
        "answer": """
        🚀👨‍🚀🌌
        """,
    },
]

memory = ConversationSummaryBufferMemory(
    llm=llm,
    max_token_limit=120,
    return_messages=True,
)


example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What emoji can describe this {movie}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt=example_prompt,
    examples=examples,
)

final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a movie expert, you give an answer with 3 emojis."),
        example_prompt,
        MessagesPlaceholder(variable_name="history"),
        ("human", "What emoji can you describe this {movie} with?"),
    ]
)

def load_memory(_):
  return memory.load_memory_variables({})["history"]

chain = RunnablePassthrough.assign(history=load_memory) | final_prompt | llm

def invoke_chain(movie):
    result = chain.invoke({"movie": movie})
    memory.save_context(
        {"input": movie},
        {"output": result.content},
    )
    print(result)
    
invoke_chain("Interstellar")
invoke_chain("Gravity")

🚀🌌⏳content='🚀🌌⏳'
🌌🌎👩‍🚀content='🌌🌎👩\u200d🚀'


In [ ]:
invoke_chain("What was the first movie I asked?")

In [ ]:
invoke_chain("What was the second movie I asked?")

In [ ]:
invoke_chain("What was the fourth movie I asked?")

In [3]:
from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory(return_messages=True, memory_key="history")

llm = ChatOpenAI(
  temperature=0.1,
)

cache_dir = LocalFileStore("./.cache/")


splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)
loader = UnstructuredFileLoader("./files/document.docx")

docs = loader.load_and_split(text_splitter=splitter)

embeddings = OpenAIEmbeddings()

cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

vectorstore = FAISS.from_documents(docs, cached_embeddings)

retriver = vectorstore.as_retriever()

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer questions using only the following context. If you don't know the answer just say you don't know, don't make it up:\n\n{context}",
        ),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{question}"),
    ]
)

def load_memory(_):
    return memory.load_memory_variables({})["history"]

chain = (
    {
        "context": retriver, 
        "question": RunnablePassthrough(),
        "history": RunnableLambda(load_memory),
    }
    | prompt 
    | llm
)

def invoke_chain(question):
    result = chain.invoke(question)
    memory.save_context(
        {'input': question},
        {'output': result.content},
    )
    print(result)

In [4]:
invoke_chain("Is Aaronson guilty?")

content='According to the document, Jones, Aaronson, and Rutherford were guilty of the crimes they were charged with.'


In [5]:
invoke_chain("What message did he write in the table?")
invoke_chain("Who is Julia?")

content='He wrote "FREEDOM IS SLAVERY" and then "TWO AND TWO MAKE FIVE" on the table.'
content='Julia is a character who was involved with Winston in the novel.'


In [3]:
aa = RunnablePassthrough()

aa.invoke({"hi":"hello"}).content

AttributeError: 'dict' object has no attribute 'content'